In [41]:
import numpy as np
import sys

In [42]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') 
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [43]:
import matplotlib.pyplot as plt
%matplotlib inline 
import networkx as nx

In [44]:
from nltk.tokenize.punkt import PunktSentenceTokenizer

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [45]:
!pip install newspaper3k
from newspaper import Article

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
def downloadcontent(url):
    # downloads the whole webpage
    article = Article(url)
    article.download()

    # parses the downloaded html
    article.parse()
    text = article.text
    return text
    


Download the content from the given URL


In [47]:
content = downloadcontent("https://www.apple.com/legal/privacy/data/en/apple-id/")

connect to google drive and create a text file with writing the content

In [48]:

with open('/content/drive/MyDrive/NLPproject/inputdocument.txt', 'w') as f:
    f.write(content)


In [49]:
def readDoc(name):
    
    print('You have asked for the document {}'.format(name))
    f = open(name, 'r')
    document = f.read()
    f.close()
    print(type(document))
    return document


In [50]:
document = readDoc('/content/drive/MyDrive/NLPproject/inputdocument.txt')
print('The length of the file is:', end=' ')
print(len(document))

You have asked for the document /content/drive/MyDrive/NLPproject/inputdocument.txt
<class 'str'>
The length of the file is: 22227


In [51]:
document


'When you sign in with your Apple ID, you will enable services such as iCloud, the App Store, iMessage, and Find My on this iPhone.\n\nCertain data, including your contacts, calendars, photos, documents, health, activity, and other app data, will be sent to Apple to store and back up on your behalf.\n\nIn addition, your device will be associated with your Apple ID to provide you with better service and support.\n\nYour Apple ID and all Apple services are designed from the ground up to protect your privacy. We work hard to collect only the data we need to make your experience better. When we do collect data, we believe it’s important for you to know what we’re collecting and why we need it, so you can make informed choices.\n\nAn Apple ID is the personal account you use to access Apple services like iCloud, the App Store and other Apple online stores, iMessage, and FaceTime, and to access your content across all your devices and the web. Your Apple ID includes the information you use to

In [52]:
def tokenize(document):
    # We are tokenizing using the PunktSentenceTokenizer
    # we call an instance of this class as sentence_tokenizer
    doc_tokenizer = PunktSentenceTokenizer()
    
    # tokenize() method: takes our document as input and returns a list of all the sentences in the document
    
    # sentences is a list containing each sentence of the document as an element
    sentences_list = doc_tokenizer.tokenize(document)
    return sentences_list

In [53]:
sentences_list = tokenize(document)

In [54]:
print(type(sentences_list))

<class 'list'>


In [55]:
for i in sentences_list:
    print(i)

When you sign in with your Apple ID, you will enable services such as iCloud, the App Store, iMessage, and Find My on this iPhone.
Certain data, including your contacts, calendars, photos, documents, health, activity, and other app data, will be sent to Apple to store and back up on your behalf.
In addition, your device will be associated with your Apple ID to provide you with better service and support.
Your Apple ID and all Apple services are designed from the ground up to protect your privacy.
We work hard to collect only the data we need to make your experience better.
When we do collect data, we believe it’s important for you to know what we’re collecting and why we need it, so you can make informed choices.
An Apple ID is the personal account you use to access Apple services like iCloud, the App Store and other Apple online stores, iMessage, and FaceTime, and to access your content across all your devices and the web.
Your Apple ID includes the information you use to sign in, as 

In [56]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 5.8 MB 33.6 MB/s 
     |████████████████████████████████| 1.3 MB 57.5 MB/s 
     |████████████████████████████████| 182 kB 65.6 MB/s 
     |████████████████████████████████| 7.6 MB 37.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=8be77dea47541a954fbf5afafcb9c376c481133fbe2ae51dea29a85fc4066c07
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [57]:
from sentence_transformers import SentenceTransformer, util

In [58]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2022-12-04 16:35:49--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-12-04 16:35:49--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-12-04 16:35:49--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

In [82]:
# remove punctuations, numbers and special characters
new_sentences = pd.Series(sentences_list).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
new_sentences = [s.lower() for s in new_sentences]

<ipython-input-82-59b81b5063e2>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  new_sentences = pd.Series(sentences_list).str.replace("[^a-zA-Z]", " ")


In [83]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [84]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [85]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [86]:
# remove stopwords from the sentences
new_sentences = [remove_stopwords(r.split()) for r in new_sentences]

In [87]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [88]:
query = ["Data Security"] * len(new_sentences)

In [89]:
# make alphabets lowercase
query = [s.lower() for s in query]


In [90]:
query_vectors = []
for i in query:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  query_vectors.append(v)

In [91]:
print(len(query_vectors))

140


In [69]:
sentence_vectors = []
for i in new_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [70]:
print(len(sentence_vectors))

140


In [71]:
# similarity matrix
# sim_mat = np.zeros([len(sentences_list), len(sentences_list)])

In [72]:
sim_query = np.zeros([len(query), len(sentences_list)])

In [73]:
from sklearn.metrics.pairwise import cosine_similarity

In [74]:
# for i in range(len(sentences_list)):
#   for j in range(len(sentences_list)):
#     if i != j:
#       sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [75]:
for i in range(len(query)):
  for j in range(len(sentences_list)):
    if i != j:
      sim_query[i][j] = cosine_similarity(query_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [76]:
import networkx as nx

# nx_graph = nx.from_numpy_array(sim_mat)
# scores = nx.pagerank(nx_graph)

nx_graph = nx.from_numpy_array(sim_query)
scores = nx.pagerank(nx_graph)

In [77]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences_list)), reverse=True)
# Extract top sentences as the summary
final_sentences = []
for i in range(20):
  final_sentences.append(ranked_sentences[i][1])

In [78]:
for each in final_sentences:
  print(each)

This data may include:

Contacts

Calendars

Reminders

Bookmarks

Safari tabs

Health data

Home data

Notes

Photos

Documents

Wallet data

Keychain and passwords

Device and account settings

Data from third-party apps that use iCloud

iCloud Backup is also enabled to help ensure you have a current backup of the data on your device in case you need to replace or restore it.
Your Apple ID includes the information you use to sign in, as well as all the contact, payment, and security details, such as trusted phone numbers or security questions, that you use across Apple services.
Certain data, including your contacts, calendars, photos, documents, health, activity, and other app data, will be sent to Apple to store and back up on your behalf.
Your Apple ID account information will be used with each service, and certain data from your device, including your selected profile photo, your contacts, calendars, photos, documents, health, activity, Safari tabs, and other app data, will be se

In [79]:
len(final_sentences)

20

In [80]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [81]:
model = SentenceTransformer('nq-distilbert-base-v1')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/540 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/554 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [97]:
def extractSentencesFromText(text):
    sections = text.replace('\n',' ').replace('\r',' ').split('.')
    passages = []
    for section in sections:
        passages.append(['', removeUnicode(section.strip())])
    return passages

def removeUnicode(text):
    string_encode = text.encode("ascii", "ignore")
    return string_encode.decode()

def scorePassages(query):
    query_embedding = model.encode(query)
    scores = util.pytorch_cos_sim(query_embedding, passage_embedding).tolist()[0]
    score_dict = {}
    sentence_counter = 0
    for score in scores:
        score_dict[sentence_counter] = score
        sentence_counter+=1
    return score_dict

In [98]:
type(document)

str

In [99]:
document

'When you sign in with your Apple ID, you will enable services such as iCloud, the App Store, iMessage, and Find My on this iPhone.\n\nCertain data, including your contacts, calendars, photos, documents, health, activity, and other app data, will be sent to Apple to store and back up on your behalf.\n\nIn addition, your device will be associated with your Apple ID to provide you with better service and support.\n\nYour Apple ID and all Apple services are designed from the ground up to protect your privacy. We work hard to collect only the data we need to make your experience better. When we do collect data, we believe it’s important for you to know what we’re collecting and why we need it, so you can make informed choices.\n\nAn Apple ID is the personal account you use to access Apple services like iCloud, the App Store and other Apple online stores, iMessage, and FaceTime, and to access your content across all your devices and the web. Your Apple ID includes the information you use to

In [100]:
passages = extractSentencesFromText(document)
print ('Total Passages:', len(passages))
passages[:3]

Total Passages: 154


[['',
  'When you sign in with your Apple ID, you will enable services such as iCloud, the App Store, iMessage, and Find My on this iPhone'],
 ['',
  'Certain data, including your contacts, calendars, photos, documents, health, activity, and other app data, will be sent to Apple to store and back up on your behalf'],
 ['',
  'In addition, your device will be associated with your Apple ID to provide you with better service and support']]

In [101]:
test=[ i[0]!=i[1] for i in passages]

In [102]:
test=[]
for i in passages:
  if(i[0]!=i[1]):
    test.append(i)

In [103]:
print ('Total Passages:', len(test))
test[:3]

Total Passages: 154


[['',
  'When you sign in with your Apple ID, you will enable services such as iCloud, the App Store, iMessage, and Find My on this iPhone'],
 ['',
  'Certain data, including your contacts, calendars, photos, documents, health, activity, and other app data, will be sent to Apple to store and back up on your behalf'],
 ['',
  'In addition, your device will be associated with your Apple ID to provide you with better service and support']]

In [104]:
passage_embedding = model.encode(test)
print ('Passage Embeddings:')
print (passage_embedding[:3])

Passage Embeddings:
[[-0.0130983   0.70132977  0.0967797  ...  0.38090378 -0.21070716
  -0.18153597]
 [ 0.5007283   0.58102477  0.13659434 ... -0.247536   -0.8001621
  -0.6996776 ]
 [-0.01934662  0.8955776   0.33124062 ... -0.18706158 -0.394461
  -0.78111815]]


In [107]:
query = "Is my location being shared?"

score_dict = scorePassages(query)

for w in sorted(score_dict, key=score_dict.get, reverse=True):
    print(score_dict[w], test[w][1])

0.6366816759109497 Share My Location is enabled only when you actively choose to share your location with someone else
0.5795108079910278 When you have Location Services enabled, you can also choose to share your devices current location using Share My Location
0.4856724441051483 You can manage or turn off location sharing with individual family members in Find My settings on your device
0.46178820729255676 If you choose to share your location with your family, all family members will automatically be able to see the location of all your devices in the Find My app so they can help you locate a missing device or see your current location on a map
0.45096254348754883 Your first and last name associated with your account and the file name will be available to anyone who has access to the sharing link, regardless of whether your share is private or public
0.42495447397232056 If you use this feature and both the device used for your location and the friends device you are sharing your locat